<h1><b>Solving the Travelling Salesman Problem (TSP) by the Genetic Algorithm (GA)</b></h1>
<h2><b>The Travelling Salesman Problem (TSP)</b></h2>
<p1>It is defined as:

<i>Given a set of waypoints and distances between them, find the shortest possible route that visits each waypoint once, then return to the initial waypoint</i>

</p1>

The TSP is a minimization problem, and the objective function is stated as the following:

\begin{align}
Minimize:
\sum_{i=1}^{n} \sum_{j=1}^{n} d_{ij} x_{ij}
\end{align}

where 

In [40]:
"""Import Libraries"""
import csv
import googlemaps
import math
import pandas as pd
import random
import time

In [2]:
"""Dataset of Station Address and Coordinates"""
geo_data = pd.read_csv("50_Landmarks.csv") 
stations = geo_data["Station"]

In [ ]:
"""Setup of Google Maps API"""


In [3]:
"""Distance Matrix of """
dm_data = "50_Landmarks_Distance_Matrix.csv"

with open(dm_data, newline="") as csvfile: # Read and convert distance matrix into a 2D list
    reader = csv.reader(csvfile, quoting=csv.QUOTE_NONNUMERIC)
    dis_mtx = list(reader)

In [36]:
class GA:

    def __init__(self, distance_matrix, population_size, num_generations, mutation_rate):  # Initialize data structures and parameters
        self.dm = distance_matrix
        self.pop = population_size
        self.generations = num_generations
        self.mutate = mutation_rate
        self.stations = list(range(len(distance_matrix)))
        self.population = []
        self.pool = []
        self.best = ()


    def generate_routes(self):
        stations = self.stations[:]
        for i in range(self.pop):
            random.shuffle(stations)
            self.population.append((stations[:], self.total_distance(stations[:])))


    def total_distance(self, route):
        length = 0
        for i in range(len(self.dm)):
            node_1 = route[i-1]
            node_2 = route[i]
            length += self.dm[node_1][node_2]
        return length

        
    def selection(self):
        sorted_population = sorted(self.population, key=lambda x: x[1], reverse=False)
        self.pool = sorted_population[:int(math.ceil(self.pop / 10))]
        self.best = self.pool[0]
        self.population = self.pool[:]

    
    def mutation(self):
        for candidate in self.population:
            
            for k in range(self.mutate):
                route = candidate[0][:]
                start_index = random.randint(0, len(route) - 1)
                length = random.randint(2, len(self.dm) // 2)

                subset = route[start_index:start_index + length]
                route = route[:start_index] + route[start_index + length:]

                insert_index = random.randint(0, len(route) + len(subset) - 1)
                route = route[:insert_index] + subset + route[insert_index:]
                self.pool.append((route, self.total_distance(route)))
                
        self.population = self.pool[:]
        self.pool = []


    def run(self):
        start_time = time.time()
        self.generate_routes()
        for k in range(self.generations):
            self.selection()
            self.mutation()
        
        converted_route = [stations[index] for index in self.best[0]]
        end_time = time.time()
        time_diff = end_time - start_time
        return converted_route, self.best[1], time_diff

In [39]:
ga = GA(distance_matrix=dis_mtx, population_size=100, num_generations=5000, mutation_rate=9)
route, total_distance, computational_time = ga.run()
print(f"The best route found: {route}")
print(f"The total distance: {total_distance}m")
print(f"Time consumed: {computational_time}s")

The best route found: ['Chickasaw National Recreation Area, 1008 W 2nd St, Sulphur, OK 73086', 'The Alamo, Alamo Plaza, San Antonio, TX', 'Carlsbad Caverns National Park, Carlsbad, NM', 'Cable Car Museum, 94108, 1201 Mason St, San Francisco, CA 94108', 'Bryce Canyon National Park, Hwy 63, Bryce, UT', 'USS Alabama, Battleship Parkway, Mobile, AL', 'Hoover Dam, NV', 'Toltec Mounds, Scott, AR', 'San Andreas Fault, San Benito County, CA', 'Hanford Site, Benton County, WA', 'Columbia River Gorge National Scenic Area, Oregon', 'Craters of the Moon National Monument & Preserve, Arco, ID', 'Yellowstone National Park, WY 82190', 'Glacier National Park, West Glacier, MT', 'Fort Union Trading Post National Historic Site, Williston, North Dakota 1804, ND', 'Mount Rushmore National Memorial, South Dakota 244, Keystone, SD', 'Ashfall Fossil Bed, Royal, NE', 'Fort Snelling, Tower Avenue, Saint Paul, MN', 'Terrace Hill, Grand Avenue, Des Moines, IA', 'Taliesin, County Road C, Spring Green, Wisconsin',